# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [67]:
# Do all imports and installs here
import pandas as pd
import pyspark.sql.functions as sfunc
from pyspark.sql.functions import udf, col, date_add, lit, to_date, when, sum as sql_sum



### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [5]:
# Read in the data here




In [6]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,NaN,NaN,...,U,NaN,1979.0,10282016,NaN,NaN,NaN,1.897628e+09,NaN,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,NaN,1991.0,D/S,M,NaN,NaN,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,NaN,M,1961.0,09302016,M,NaN,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,1988.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,NaN,M,2012.0,09302016,NaN,NaN,AA,9.246846e+10,00199,B2


In [4]:
	
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_spark =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')


In [5]:
#write to parquet
#df_spark.write.parquet("sas_data")
df_spark=spark.read.parquet("sas_data")

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In [81]:
#gather I94 SAS data
df_sas=spark.read.parquet("sas_data")




In [82]:
# add column arrdate_int to facilitate transform to date
# add column diff_date as literal of 1960-01-01 to transform arr_date
df_sas = df_sas.withColumn('arrdate_int', df_sas.arrdate.cast('integer')) \
    .withColumn('diff_date', to_date(lit('1960-01-01'))) \
    .withColumn('gender_reported', when(df_sas.gender.isNotNull()==1, 1).otherwise(0)) \
    .withColumn('gender_male', when(df_sas.gender=="M", 1).otherwise(0)) \
    .withColumn('gender_female', when(df_sas.gender=="F", 1).otherwise(0)) \
    .withColumn('gender_other', when(df_sas.gender=="M", 0).when(df_sas.gender=="F", 0).when(df_sas.gender.isNull()==1, 0).otherwise(1)) \
    .withColumn('age_reported', when(df_sas.i94bir.isNotNull()==1, 1).otherwise(0)) \
    .withColumn('age_0_11', when(df_sas.i94bir <= 11, 1).otherwise(0)) \
    .withColumn('age_12_17', when(df_sas.i94bir.between(12, 17)==1, 1).otherwise(0)) \
    .withColumn('age_18_24', when(df_sas.i94bir.between(18, 24)==1, 1).otherwise(0)) \
    .withColumn('age_25_34', when(df_sas.i94bir.between(25, 34)==1, 1).otherwise(0)) \
    .withColumn('age_35_44', when(df_sas.i94bir.between(35, 44)==1, 1).otherwise(0)) \
    .withColumn('age_45_54', when(df_sas.i94bir.between(45, 54)==1, 1).otherwise(0)) \
    .withColumn('age_55_64', when(df_sas.i94bir.between(55, 64)==1, 1).otherwise(0)) \
    .withColumn('age_65_74', when(df_sas.i94bir.between(65, 74)==1, 1).otherwise(0)) \
    .withColumn('age_75p', when(df_sas.i94bir >= 75, 1).otherwise(0)) \
    .withColumn('arrival_type_reported', when(df_sas.i94mode.isNull()==1, 0).when(df_sas.i94mode==9, 0).otherwise(1)) \
    .withColumn('air_arrival', when(df_sas.i94mode ==1, 1).otherwise(0)) \
    .withColumn('sea_arrival', when(df_sas.i94mode ==2, 1).otherwise(0)) \
    .withColumn('land_arrival', when(df_sas.i94mode ==3, 1).otherwise(0)) \
    .withColumn('visa_type_reported', when(df_sas.i94visa ==1, 1).when(df_sas.i94visa ==2, 1).when(df_sas.i94visa ==3, 1).otherwise(0)) \
    .withColumn('business_visa', when(df_sas.i94visa ==1, 1).otherwise(0)) \
    .withColumn('pleasure_visa', when(df_sas.i94visa ==2, 1).otherwise(0)) \
    .withColumn('student_visa', when(df_sas.i94visa ==3, 1).otherwise(0)) \
    



In [83]:
# add column arrival_date as diff_date plus arrdate_int 
df_sas = df_sas.selectExpr("*", "date_add(diff_date, arrdate_int) as arrival_date", "cast(count as int) as traveler_count", "cast(i94cit as int) as i94_cit",
                          "cast(i94res as int) as i94_res", "i94port as i94_port", "i94addr as i94_addr")



In [84]:
df_sas = df_sas.select([df_sas.arrival_date, df_sas.i94_cit, df_sas.i94_res, df_sas.i94_port, df_sas.gender_reported, df_sas.gender_male, 
                        df_sas.gender_female, df_sas.gender_other, df_sas.age_reported, df_sas.age_0_11, df_sas.age_12_17, df_sas.age_18_24,
                        df_sas.age_25_34, df_sas.age_35_44, df_sas.age_45_54, df_sas.age_55_64, df_sas.age_65_74, df_sas.age_75p,
                        df_sas.arrival_type_reported, df_sas.air_arrival, df_sas.sea_arrival, df_sas.land_arrival, 
                        df_sas.visa_type_reported, df_sas.business_visa, df_sas.pleasure_visa, df_sas.student_visa, 
                        df_sas.i94_addr, df_sas.traveler_count ])

In [85]:
df_sas.printSchema()

root
 |-- arrival_date: date (nullable = true)
 |-- i94_cit: integer (nullable = true)
 |-- i94_res: integer (nullable = true)
 |-- i94_port: string (nullable = true)
 |-- gender_reported: integer (nullable = false)
 |-- gender_male: integer (nullable = false)
 |-- gender_female: integer (nullable = false)
 |-- gender_other: integer (nullable = false)
 |-- age_reported: integer (nullable = false)
 |-- age_0_11: integer (nullable = false)
 |-- age_12_17: integer (nullable = false)
 |-- age_18_24: integer (nullable = false)
 |-- age_25_34: integer (nullable = false)
 |-- age_35_44: integer (nullable = false)
 |-- age_45_54: integer (nullable = false)
 |-- age_55_64: integer (nullable = false)
 |-- age_65_74: integer (nullable = false)
 |-- age_75p: integer (nullable = false)
 |-- arrival_type_reported: integer (nullable = false)
 |-- air_arrival: integer (nullable = false)
 |-- sea_arrival: integer (nullable = false)
 |-- land_arrival: integer (nullable = false)
 |-- visa_type_reported: 

In [86]:
df_sas_grp = df_sas.groupBy("arrival_date", "i94_cit", "i94_res", "i94_port", "gender_reported", "gender_male", "gender_female", "gender_other",
                           "age_reported", "age_0_11", "age_12_17", "age_18_24", "age_25_34","age_35_44", "age_45_54", "age_55_64","age_65_74", "age_75p",
                           "arrival_type_reported", "air_arrival", "sea_arrival", "land_arrival", "visa_type_reported",
                           "business_visa", "pleasure_visa","student_visa", "i94_addr").agg(sql_sum("traveler_count").alias("traveler_count"))

In [87]:
df_sas_grp.show()

+------------+-------+-------+--------+---------------+-----------+-------------+------------+------------+--------+---------+---------+---------+---------+---------+---------+---------+-------+---------------------+-----------+-----------+------------+------------------+-------------+-------------+------------+--------+--------------+
|arrival_date|i94_cit|i94_res|i94_port|gender_reported|gender_male|gender_female|gender_other|age_reported|age_0_11|age_12_17|age_18_24|age_25_34|age_35_44|age_45_54|age_55_64|age_65_74|age_75p|arrival_type_reported|air_arrival|sea_arrival|land_arrival|visa_type_reported|business_visa|pleasure_visa|student_visa|i94_addr|traveler_count|
+------------+-------+-------+--------+---------------+-----------+-------------+------------+------------+--------+---------+---------+---------+---------+---------+---------+---------+-------+---------------------+-----------+-----------+------------+------------------+-------------+-------------+------------+--------+--

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.